In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import glob
import seaborn as sns
import skimage.io as io
from skimage import data_dir
import pandas as pd
from PIL import Image
#import split_folders
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from skimage.io import imread_collection
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

#this function reads in images and then flattens them into a dataframe
def flat_images(path):
    images=io.ImageCollection(path)
    #images=Image.open(path)
    flat_images=[]
    for i in range(len(images)):
        img=images[i]
        img1=img.flatten()
        #img1=np.asarray(img1)
        print(img1.shape)
        flat_images.append(img1)
    df=pd.DataFrame(flat_images)
    return df

#flattens ditylum images
path = r'C:\Users\smr6743\Desktop\PrePrecS20\data\1_ditylum\1\*.tif'
df_1=flat_images(path)
df_1=df_1.fillna(0)
df_1['label']=1

print('Ditylum images processed')

#flattens non-ditylum images
path1= r'C:\Users\smr6743\Desktop\PrePrecS20\data\0_notditylum\0\*.tif'
df_0=flat_images(path1)
df_0=df_0.fillna(0)
df_0['label']=0

print('Non-Ditylum images processed')

#puts both sets of images in same dataframe
df=df_1.append(df_0,sort=True)
df.reset_index(inplace=True,drop=True)
df=df.fillna(0)

print('Dataframe of all flattened images created')

# create test and train data (80/20 split)
X=df.drop(['label'],axis=1)
y=df.label

print('X and y are initialized')

(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(250000,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(59800,)
(51220,)
(48100,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(64480,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(58760,)
(55900,)
(67600,)
(48880,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(47840,)
(43680,)
(67600,)
(50700,)
(67600,)
(46280,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)
(67600,)


KeyboardInterrupt: 

In [3]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=.2,random_state=42)

In [4]:
pca=PCA()
pca.fit(X_train)
csum=np.cumsum(pca.explained_variance_ratio_)
d=np.argmax(csum>=.95)+1
print('Reduces shape to',d)
pca=PCA(n_components=d)
# reduce test set too for metrics later - not in the model itself
X_train_reduced=pca.fit_transform(X_train)
X_test_reduced=pca.transform(X_test)
print(X_train.shape, X_train_reduced.shape)

Reduces shape to 388
(543, 547600) (543, 388)


## MLP Classifier - multilayer perceptron - feed forward neural net based on logistic regression

In [5]:
# set parameters to perform a grid search and MLP classifier
params={'max_iter':[1000],'alpha':np.linspace(0.0085, 0.009, 2),'hidden_layer_sizes':[(100,100,100),(64,64,64), (32,32,32), (256,256,256),(128,128,128)],'random_state':[42]}
grid=GridSearchCV(MLPClassifier(),params,refit=True,verbose=0,cv=5)
grid.fit(X_train_reduced,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_iter=200,
                                     momentum=0.9, n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_sta...
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([0.0085, 0.009 ]),
                         '

In [6]:
# predicted based off of the model
y_pred=grid.predict(X_test_reduced)

In [7]:
# metrics
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

0.7941176470588235
[[48 10]
 [18 60]]
              precision    recall  f1-score   support

           0       0.73      0.83      0.77        58
           1       0.86      0.77      0.81        78

    accuracy                           0.79       136
   macro avg       0.79      0.80      0.79       136
weighted avg       0.80      0.79      0.80       136



In [8]:
grid.best_params_,grid.best_score_

({'alpha': 0.009,
  'hidden_layer_sizes': (256, 256, 256),
  'max_iter': 1000,
  'random_state': 42},
 0.7550644567219152)

## MLP Classifier - Scaled First

In [ ]:
# Apply transform to both the training set and the test set.
X_train_scaled = scale(X_train)
X_test_scaled = scale(X_test)
# perform PCA
pca=PCA()
pca.fit(X_train)
csum=np.cumsum(pca.explained_variance_ratio_)
d=np.argmax(csum>=.95)+1
print('Reduces shape to',d)
pca=PCA(n_components=d)
# reduce test set too for metrics later - not in the model itself
X_train_reduced=pca.fit_transform(X_train_scaled)
X_test_reduced=pca.transform(X_test_scaled)
print(X_train.shape, X_train_reduced.shape)

In [ ]:
# set parameters to perform a grid search and MLP classifier
params={'max_iter':[1000],'alpha':np.linspace(0.0085, 0.009, 2),'hidden_layer_sizes':[(100,100,100),(64,64,64), (32,32,32), (256,256,256),(128,128,128)],'random_state':[42]}
grid=GridSearchCV(MLPClassifier(),params,refit=True,verbose=3,cv=5)
grid.fit(X_train_reduced,y_train)
# predicted based off of the model
y_pred=grid.predict(X_test_reduced)

In [ ]:
# metrics
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

## Basic Logistic Regression - w/ only PCA not scaling

In [ ]:
logReg = LogisticRegression()
logReg.fit(X_train_reduced,y_train)
y_pred=logReg.predict(X_test_reduced)

# metrics
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))